In [81]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [61]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

url = 'https://www.bilbasen.dk/brugt/bil?includeengroscvr=true&includeleasing=false'

page = requests.get(url, headers=headers)
print(page)
# print(page.text)
soup = BeautifulSoup(page.text, 'html.parser')

<Response [200]>


In [62]:
# get links to all cars on page

def getCarLinks(tree):
    l = [a['href'] for a in tree.select("article > a[class^=Listing_link]")]
    return l

links = getCarLinks(soup)

links

['https://www.bilbasen.dk/brugt/bil/nissan/qashqai/12-dig-t-115-acenta-5d/6039416',
 'https://www.bilbasen.dk/brugt/bil/mazda/3/20-e-skyactiv-g-150-sky-style-aut-5d/5883516',
 'https://www.bilbasen.dk/brugt/bil/mini/clubman-cooper-s/20-aut-6d/6052224',
 'https://www.bilbasen.dk/brugt/bil/skoda/superb/15-tsi-150-style-dsg-5d/5912584',
 'https://www.bilbasen.dk/brugt/bil/hyundai/i20/10-t-gdi-essential-5d/6079058',
 'https://www.bilbasen.dk/brugt/bil/mercedes/e220-d/20-amg-line-stc-aut-5d/6058914',
 'https://www.bilbasen.dk/brugt/bil/vw/eos/20-tdi-2d/6005142',
 'https://www.bilbasen.dk/brugt/bil/opel/grandland-x/12-t-130-elegance-aut-5d/6079061',
 'https://www.bilbasen.dk/brugt/bil/vw/polo/15-tsi-150-r-line-dsg-5d/6086076',
 'https://www.bilbasen.dk/brugt/bil/mercedes/gle53/30-amg-coup-aut-4matic-5d/6071081',
 'https://www.bilbasen.dk/brugt/bil/audi/a3/16-tdi-110-ambiente-3d/6018349',
 'https://www.bilbasen.dk/brugt/bil/mercedes/a200-d/20-amg-line-aut-5d/5928526',
 'https://www.bilbasen.d

In [63]:
# get single car page

def getCarSoup(link):
    page = requests.get(link, headers=headers)
    if page.ok:
        soup = BeautifulSoup(page.text, 'html.parser')
        # print(page)
        return soup
    else:
        print(page.reason)

carPage = getCarSoup(links[0])
# carPage  
print(carPage)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><link href="https://www.bilbasen.dk/favicon.ico" rel="shortcut icon"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="PmzU04GwUWp-nFvo0avXO6pUm6bEFP8VjumHzThcbhE" name="google-site-verification"/><meta content="nositelinkssearchbox" name="google"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><link href="//i.ebayimg.com" rel="preconnect"/><link href="//www.googletagmanager.com" rel="preconnect"/><link href="//www.google-analytics.com" rel="preconnect"/><link href="//www.google.com" rel="preconnect"/><link as="font" crossorigin="anonymous" href="/Content/fonts/GTWalsheimProRegular/GTWalsheimProRegular.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" href="/Content/fonts/GTWalsheimProMedium/GTWalsheimProMedium.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" href="/Conten

In [83]:
# get car data

def extractCarData(soupTree):

    model = soupTree.select_one('h1[data-e2e="car-make-model-variant"]')['title']
    price = soupTree.select_one('span[data-e2e="car-retail-price"]').text

    detail_div = soupTree.select("div[aria-label=Detaljer] tr")
    # print(detail_div)
    # print(len(detail_div))
    details = {}
    for row in detail_div:
        # print( row.find_all(['th', 'td']) )
        key = row.select_one('th[role=cell][scope=row]').text
        value = row.select_one('td').text
        details[key] = value
        
    description = soupTree.select_one("div[aria-label=beskrivelse] p")
    # print(description)
    # description.p.unwrap()

    
    model_div = soupTree.select("div[aria-label='Generelle modeloplysninger*'] tr")
    # print(model_div)
    # print(len(model_div))
    model = {}
    for row in model_div:
        # print( row.find_all(['th', 'td']) )
        key = row.select_one('th[role=cell][scope=row]').text
        value = row.select_one('td').text
        model[key] = value

    # "Udstyr og tilbehør"
    equipment_div = soupTree.select("div[aria-label='Udstyr og tilbehør'] tr")
    # print(equipment_div)
    # print(len(equipment_div))
    equipment = {}
    for row in equipment_div:
        # print( row.find_all(['th', 'td']) )
        key = row.select_one('th[role=cell][scope=row]').text
        if row.select_one('td'):
            value = row.select_one('td').text
            equipment[key] = value
        else:
            equipment[key] = None

    data = {}
    data['model'] = model
    data['price'] = price
    data['details'] = details
    data['description'] = description
    data['model_details'] = model
    data['equipment'] = equipment
    data['collected'] = datetime.now().isoformat()
    return data



data = extractCarData(carPage)
data

{'model': {'Nypris': '249.989 kr.',
  'Kategori': 'Personbil',
  'Type': 'CUV',
  'Bagagerumsstørrelse': '-',
  'Vægt': '1.318 kg',
  'Bredde': '181 cm',
  'Længde': '439 cm',
  'Højde': '159 cm',
  'Lasteevne': '542 kg',
  'Max. trækvægt m/bremse': '1.200 kg',
  'Trækhjul': 'Forhjulstræk',
  'Cylindre': '4',
  'ABS-bremser': 'Ja',
  'ESP': 'Ja',
  'Airbags': '6',
  'Tankkapacitet': '55 l',
  'Døre': '5'},
 'price': '159.900 kr.',
 'details': {'Modelår': '2018',
  '1. registrering': '1/2018',
  'Kilometertal': '88.000 km',
  'Drivmiddel': 'Benzin',
  'Brændstofforbrug': '(NEDC) 17,9 km/l',
  'CO2 udledning': '129 g/km',
  'Euronorm': '-',
  'Periodisk afgift': '2.780 kr. / år',
  'Ydelse': '115 hk/190 nm',
  'Acceleration': '10,6 sek.',
  'Tophastighed': '185 km/t',
  'Geartype': 'Manuel',
  'Antal gear': '6',
  'Trækvægt': '1.200 kg',
  'Farve': 'Metal'},
 'description': <p>alu., fuldaut. klima, fjernb. c.lås, fartpilot, sædevarme, højdejust. førersæde, 4x el-ruder, el-spejle, parkeri

In [65]:
# get link to next page

base_url = 'http://bilbasen.dk'

def getNextPage(tree):
    return base_url + tree.select_one("div a[data-e2e=pagination-next]")['href']
    
next = getNextPage(soup)

next

'http://bilbasen.dk/brugt/bil?includeengroscvr=true&includeleasing=false&page=2'

In [66]:
# loop pages